### O que veremos hoje?

Hoje avançaremos um pouco mais em nossos conhecimentos sobre a estrutura de um game. Mais especificamente, utilizaremos OO (Orientação a Objetos) para criar 2 telas de usuário, as quais:
* Tela e boas vindas (carregamento)
* Menu principal
* Tela de fim de jogo (encerramento)

Pegue um café, fique confortável e embarque conosco nessa jornada de conhecimento.

### Introdução (bem por alto) sobre OO

Orientação a Objetos (OO) é um paradigma de programação onde separamos cada parte do nosso código em classes autocontidas, que possuem métodos (funções internas a uma classe) e atributos (variáveis internas a uma classe). Ao instaciarmos uma classe, criamos um novo objeto. A divisão de tarefas a diferentes objetos torna o código mais organizado e mais fácil de ser analisado, tornando projetos grandes bem melhor administrados. 

A OO permite o conceito de herança, que é quando classes podem herdar a mesma estrutura de outras classes. Isso torna o projeto mais padronizado e evita a cópia desnecessária (pleonasmo?) de código. Por exemplo, as telas de um game possuem métodos e atributos em comum. Por exemplo, elas precisam:
* ter um método para iniciar um temporizador ao ser iniciada para o caso de eventos no tempo; 
* ler ou bloquear a entrada do usuário. Telas de carregamento bloqueiam a entrada, enquanto menus lêem;
* enviar o sinal de próxima tela para um gerenciador de telas, quando encerrada;

Em nosso projeto, criaremos uma classe chamada _UserScreen_. Todas as demais telas herdarâo determinada estrutura dessa classe. Além disse, teremos uma classe que gerenciará, entre outras coisas, a troca entre as telas - chamada _GameManeger_. Atributos gerais do game, e que podem ser usadas em quaisquer das telas, estarão localizadas nessa tela. Exemplos desses atributos são:
* Todas as telas possíveis do game
* Tela atual e estado do game (localização do player no mapa, estado dos inimigos)
* Estado atual do jogador (vida, mana, pontuação atual)
* Histórico de pontuações máxima

Neste tutorial faremos a estrutura de telas (o que já é bastante coisa). Nos próximos cuidaremos com mais detalhe sobre os demais detalhes.

### Estrutura básica do _GameManager_

Diferentemente dos tutoriais anteriores, onde dávamos o código inicialmente e depois comentávamos, neste tutorial nós começaremos do zero. Se quiser checar o código completo, o mesmo se encontra ao fim deste notebook. 

Agora criaremos nosso objeto _GameManager_, bem como seu construtor (o primeiro método que é chamado quando uma classe é instanciada num novo objeto). Lembre: em Python, as classes são definidas em _CamelCase_, enquanto os métodos e atributos são definidos em snake_case.

In [1]:
import pygame
from pygame.locals import *

class GameManager(object):
    # Construtor da classe GameManager
    def __init__(self):
        pygame.init()                   # Inicializa o Pygame
        self.fps = 60                   # FPS do game
        self.fps_clock = None           # Conterá o relógio de clock do Pygame
        self.window_size = (640, 480)   # Resolução da janela
        self.window_name = 'OO Game'    # Nome da janela
        self.screen_dictionary = {}     # Dicionário com as possíveis telas
        self.current_screen_key = ''    # String com o nome da tela ativa    
        self.previous_screen_key = ''   # String com o nome da última tela ativa
        self.main_surface = None        # Surface da janela principal
        self.keep_running = False       # Booleano que indica o estado do game
        
    # Método para inicializar o game
    def start(self):
        self.fps_clock = pygame.time.Clock()
        self.main_surface = pygame.display.set_mode(self.window_size)
        pygame.display.set_caption(self.window_name)
        self.keep_running = True
        
        """
        self.current_screen_key = 'START_SCREEN'
        """
        while self.keep_running:
            """ 
            self.screen_dictionary[self.current_screen_key].update()
            self.screen_dictionary[self.current_screen_key].draw()
            """
            pygame.display.update() 
            self.fps_clock.tick(self.fps)    
            
        self.end()
        
    # Metódo para parar o game
    def stop(self):
        self.keep_running = False
    
    # Método para fechar o game
    def end(self):
        pygame.quit()

    # Método para desenhar na main_window
    def draw(self, surface):
        self.main_surface.blit(surface, (0,0))

    # Método para cadastro de telas
    def register_screen(self, new_screen):
        """
        self.screen_dictionary[new_screen.screen_key] = new_screen
        """
        
    # Método para troca de telas
    def change_current_screen(self, next_screen_key):
        pass

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


Note que há quatro seções de código comentado entre aspas-duplas. Isso se deve ao fato de ainda não possuírmos nossa tela cadastrada. O que acabamos de criar aqui foi nosso _GameManager_ que será o responsável por iniciar as configurações globais do game, bem como a estrutura necessária para o gerenciamento de telas. Parte dos códigos utilizados aqui já foi discutida nos tutoriais anteriores.

É imporante notar que o método ```GameManager.__init__``` é o construtor da classe _GameManager_. Ele é o primeiro método a ser chamado quando uma classe é instanciada. Note também que o ```__init__``` e todos os demais métodos possuem, obrigatoriamente, o parâmetro _self_ como parâmetro inicial. Isso é nada mais que uma referência do método ao objeto que ele pertence. Dessa forma, você pode ter acesso aos outros métodos e atributos dentro do escopo do método atual. Dê uma lida novamente no código, e veja se agora esse monte de ```self``` ficou mais claro xD

Ótimo você continua por aqui! Vamos falar sobre os códigos comentados... Eles foram uma espécie de ponto de partida para o próximo objeto que devemos criar: a classe _Screen_. Ao ler o código acima atenciosamente, notamos que ```self.screen_dictionary``` é um dicionário que possui _Strings_ como chaves e _Screen_ como valores. Vemos também que há dois atributos _String_ que irão armazenar chaves desse nosso dicionário no futuro. Notamos, também, que haverá pelo menos dois métodos na classe _Screen_: ```update()``` e ```draw()```. O primeiro será responsável por atualizar o estado da tela. O segundo será o responsável por desenhar toda a tela na nossa _Surface_ principal: a _GameManager.main_surface_. 

Eu sei... Eu sei... Tem bastante coisa até agora. Mas tudo vai fazer sentido quando começarmos a implementar. Mão na massa!

### Estrutura básica da classe pai _Screen_

Vamos agora aprender um pouco sobre herança. O Python não é a melhor linguagem para se aprender tal assunto, mas é possível pegar a ideia. Podemos utilizar uma classe pai para abrigar métodos e atributos comuns às classes filhas. É possível, dessa forma, reutilizar o mesmo código para várias classes filhas. 

Uma outra vantagem é a padronização das classes filhas. É possível criar interfaces na classe pai, de forma que a assinatura dos métodos da classe filha (o nome dos métodos e seus parâmetros) sigam determinado padrão. Como definimos acima, já temos dois padrões a serem seguidos por todas as classes que herdam da nossa _Screen_: ```update()``` e ```draw()```.

Vamos, então, criar nossa classe pai _Screen_. Por agora, pode colocar no mesmo arquivo do código anterior. Vamos seperá-los mais pra frente:

In [2]:
class Screen(object):
    def __init__(self, screen_key, game_manager):
        self.game_manager = game_manager # GameManager      
        self.screen_key = screen_key     # String que representa a chave da tela
        self.screen_surface = pygame.Surface(game_manager.window_size) # Surface onde gráficos serão desenhados
        self.start_ticks = pygame.time.get_ticks()   # Ticks capturados na criação da tela.
        self.current_ticks = 0 # Ticks capturados no momento atual. Atualizado a cada update.
        self.current_time = 0 # Tempo em segundos desde a criação da tela. Atualizado a cada update.
        self.events = [] # Conterá a lista de eventos, já que o pygame.event.get() limpa a fila
    
    # Deve ser chamado antes do método filho, pois atualiza o timer
    def update(self):
        self.current_ticks = pygame.time.get_ticks()
        self.current_time = (self.current_ticks - self.start_ticks) / 1000
        self.events = pygame.event.get()
        for event in self.events:
            if event.type == QUIT:
                self.game_manager.stop()
    
    # Deve ser chamado depois do método filho, pois screen_surface devem estar pronta
    def draw(self):
        self.game_manager.draw(self.screen_surface)

Podemos notar que passamos nosso GameManager como parâmetro. Tal fato é necessário, pois há parâmetros necessários nele, que utilizamos aqui. É importante, também, ressaltar a importância de utilizarmos métodos quando desejarmos alterar um atributo de outra classe. Isso torna o código mais fácil de debugar do que se alterássemos seus valores diretamente. Portanto, para lembrar:
* Ler atributos de outra classe diretamente? Tudo bem.
* Alterar atributos de outra classe diretamente? Nem pensar. Utilize métodos para isso.

Por esse motivo que criamos as funções ```GameManager.stop()``` (que altera a ```GameManager.keep_runing```) e ```game_manager.draw()``` (que altera a ```GameManager.main_surface```). 

Vamos comentar sobre o método ```Screen.update()```. Ele é o responsável por checar eventos, dentre eles a entrada do usuário. O que fizemos aqui foi programar a rotina de atualização comum a todas as telas:
* Atualizar o temporizador
* Checar se o usuário clicou em _Fechar janela_
A atualização do temporizador é feita atráves da contagem de Ticks, como já foi discutido no tutorial anterior. Dividimos a diferença por 1000 para obter o tempo em segundos, ao invés de milisegundos. A checagem da saída de game é tratada no _GameManager_, daí a chamada da função ```game_manager.stop()```. Dessa forma, é necessário chamar o ```update()``` da classe pai antes da classe filha. Veremos isso na próxma seção de código.
 
Já a função ```draw()``` simplesmente se encarrega de desenhar na surface do _GameManager_ através do método ```GameMAnager.draw()```. Assim, a cada _Surface_ filha, precisaremos nos preocupar apenas com o desenho na Surface interna.

Uma última questão importante é a questão do tratamento de eventos. Como mostrado na documentação do [pygame.event.get()](https://www.pygame.org/docs/ref/event.html), tal método consome todos os eventos presentes na fila de mensagens de eventos. Dessa forma, é necessário armazená-los numa lista para poderem ser futuramente tratados pela classe filha.

Vamos à implementação das classes filhas? Só se for agora!

### Estutura da _StartScreen_ , a primeira classe filha da _Screen_

Lembra da tela de carregamento do tutorial anterior? Faremos algo parecido. Se tiver dúvida no código, é bom dar uma olhada no Tutorial 2. Aqui vamos nos preocupar apenas com os aspectos da Orientação a Objetos. 

Como já foi dito, as classes filhas herdam todos os métodos e atributos definidos na classe pai. Para acessá-los, basta utilizar o pré-fixo ```super()``` antes do método ou atributo. Dessa forma, precisaremos criar apenas os atributos e métodos específicos para cada classe filha, aproveitando tudo o que a classe pai nos oferece. Eis nossa _StartScreen_, linda e organizada, que herda da classe _Screen_:

In [3]:
class StartScreen(Screen):
    def __init__(self, screen_key, game_manager):
        super().__init__(screen_key, game_manager)
        self.fg_color = (0, 0, 0)
        self.bg_color = (255, 255, 255)
        self.font_size = 25
        self.font_obj = pygame.font.Font('freesansbold.ttf', self.font_size)
        self.text = '.'
        self.text_surface = self.font_obj.render(self.text, True, self.fg_color, self.bg_color)
        self.text_rect = self.text_surface.get_rect()
        self.text_rect.center = self.screen_surface.get_rect().center
        pygame.time.set_timer(USEREVENT, 1000)

    def update(self):
        super().update()
        for event in self.events:
            if event.type == USEREVENT:
                if len(self.text) == 3:
                    self.text = '.'
                else:
                    self.text = self.text + '.'
        
    def draw(self):
        self.text_surface = self.font_obj.render(self.text, True, self.fg_color, self.bg_color) 
        self.screen_surface.fill(self.bg_color)
        self.screen_surface.blit(self.text_surface, self.text_rect)
        super().draw()

Note como utilizamos o constutor da classe pai dentro do contrutor da nossa classe filha:
```
class StartScreen(Screen):
    def __init__(self, screen_key, game_manager):
        super().__init__(screen_key, game_manager)
```

Note também que é possível definirmos quando iremos utilizar os métodos ```super()```. No método ```StartScreen.update()``` chamamos o método pai no inicio, pois precisamos preencher a fila de eventos. Já no método ```StartScreen.draw()```, utilizamos o método pai no fim, pois precisávamos da nossa ```screen_surface``` já desenhada.

Parabéns por chegar até aqui e não desistir. Você acaba de criar seu primeiro objeto com herança. Já podemos descomentar as linhas iniciais e testar nosso código. O código completo fica assim:

In [4]:
import pygame
from pygame.locals import *

class GameManager(object):
    # Construtor da classe GameManager
    def __init__(self):
        pygame.init()                   # Inicializa o Pygame
        self.fps = 60                   # FPS do game
        self.fps_clock = None           # Conterá o relógio de clock do Pygame
        self.window_size = (640, 480)   # Resolução da janela
        self.window_name = 'OO Game'    # Nome da janela
        self.screen_dictionary = {}     # Dicionário com as possíveis telas
        self.current_screen_key = ''    # String com o nome da tela ativa    
        self.previous_screen_key = ''   # String com o nome da última tela ativa
        self.main_surface = None        # Surface da janela principal
        self.keep_running = False       # Booleano que indica o estado do game
        
    # Método para inicializar o game
    def start(self):
        self.fps_clock = pygame.time.Clock()
        self.main_surface = pygame.display.set_mode(self.window_size)
        pygame.display.set_caption(self.window_name)
        self.keep_running = True
        
        self.current_screen_key = 'START_SCREEN'

        while self.keep_running:
            self.screen_dictionary[self.current_screen_key].update()
            self.screen_dictionary[self.current_screen_key].draw()

            pygame.display.update() 
            self.fps_clock.tick(self.fps)    
            
        self.end()
        
    # Metódo para parar o game
    def stop(self):
        self.keep_running = False
    
    # Método para fechar o game
    def end(self):
        pygame.quit()

    # Método para desenhar na main_window
    def draw(self, surface):
        self.main_surface.blit(surface, (0,0))

    # Método para cadastro de telas
    def register_screen(self, new_screen):
        self.screen_dictionary[new_screen.screen_key] = new_screen
        
        
    # Método para troca de telas
    def change_current_screen(self, next_screen_key):
        pass
    
class Screen(object):
    def __init__(self, screen_key, game_manager):
        self.game_manager = game_manager # GameManager      
        self.screen_key = screen_key     # String que representa a chave da tela
        self.screen_surface = pygame.Surface(game_manager.window_size) # Surface onde gráficos serão desenhados
        self.start_ticks = pygame.time.get_ticks()   # Ticks capturados na criação da tela.
        self.current_ticks = 0 # Ticks capturados no momento atual. Atualizado a cada update.
        self.current_time = 0 # Tempo em segundos desde a criação da tela. Atualizado a cada update.
        self.events = None # Conterá a lista de eventos, já que o pygame.event.get() limpa a fila
    
    # Deve ser chamado antes do método filho, pois atualiza o timer
    def update(self):
        self.current_ticks = pygame.time.get_ticks()
        self.current_time = (self.current_ticks - self.start_ticks) / 1000
        self.events = pygame.event.get()
        for event in self.events:
            if event.type == QUIT:
                self.game_manager.stop()
    
    # Deve ser chamado depois do método filho, pois screen_surface devem estar pronta
    def draw(self):
        self.game_manager.draw(self.screen_surface)
        
#%%

class StartScreen(Screen):
    def __init__(self, screen_key, game_manager):
        super().__init__(screen_key, game_manager)
        self.fg_color = (0, 0, 0)
        self.bg_color = (255, 255, 255)
        self.font_size = 25
        self.font_obj = pygame.font.Font('freesansbold.ttf', self.font_size)
        self.text = '.'
        self.text_surface = self.font_obj.render(self.text, True, self.fg_color, self.bg_color)
        self.text_rect = self.text_surface.get_rect()
        self.text_rect.center = self.screen_surface.get_rect().center
        pygame.time.set_timer(USEREVENT, 1000)

    def update(self):
        super().update()    # Deve ser chamado ANTES do update() filho
        for event in self.events:
            if event.type == USEREVENT:
                if len(self.text) == 3:
                    self.text = '.'
                else:
                    self.text = self.text + '.'
        
    def draw(self):
        self.text_surface = self.font_obj.render(self.text, True, self.fg_color, self.bg_color) 
        self.screen_surface.fill(self.bg_color)
        self.screen_surface.blit(self.text_surface, self.text_rect)
        super().draw()      # Deve ser chamado DEPOIS do update() pai

game_manager = GameManager()
start_screen = StartScreen('START_SCREEN', game_manager)

game_manager.register_screen(start_screen)
game_manager.start()

### O que vem por aí?

Eu sei que vimos bastante coisa hoje. Se você chegou até aqui, já estou satisfeito. Mesmo que só apareça 3 pontinhos na tela, é a estrutura por trás dele que é digna de apreciação. 

Nos próximos tutoriais criaremos duas outras classes que herdam da _Screen_, a saber: a _MenuScreen_ e a _EndScreen_. Abordaremos assuntos como a leitura de teclado do usuário e a animação com _Sprites_. Até logo!